In [1]:

    
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup, SoupStrainer
from bs4.diagnose import diagnose

from selenium.webdriver.common.action_chains import ActionChains

driver = webdriver.Chrome()
developmental = driver.get("https://iopscience.iop.org/journal/1478-3975")
developmental_content = driver.page_source
only_issues = SoupStrainer(class_='mid-tablet-half-left')
only_volumes = SoupStrainer(class_='mid-tablet-half-right')
issue_soup = BeautifulSoup(developmental_content, 'html.parser', parse_only = only_issues)
volume_soup = BeautifulSoup(developmental_content, 'html.parser', parse_only = only_volumes)

#Check to see how the soup looks and identify any useful tags

#print(issue_soup.prettify())
#print(volume_soup.prettify())

## Due to this page's structure, first we have to acces Volume, once in Volume we may check the Issues.

##First get the journal archive
journal_archive = SoupStrainer(id = 'allVolumesSelector')
journal_soup = BeautifulSoup(developmental_content, 'html.parser', parse_only = journal_archive)

journal_link = []
for link in journal_soup.find_all('option'):
    journal_link.append(link.get('value'))

##Next acces each journal and check out the issues and acces them
driver.close()

In [4]:
issue_link = []
driver = webdriver.Chrome()
for volume in journal_link:
    volume_url = 'https://iopscience.iop.org' + volume
    vol_ = driver.get(volume_url)
    volume_content = driver.page_source
    latest_issues = SoupStrainer(class_='mb-05')
    latest_issue_soup = BeautifulSoup(volume_content, 'html.parser', parse_only = latest_issues)
    
    for issue in latest_issue_soup.find_all('a'):
        issue_link.append(issue.get('href'))

driver.close()
##Next acces each issue and retrieve title and links

In [6]:
titles = []
title_link = []
driver = webdriver.Chrome()
for issue in issue_link:
    issue_url = 'https://iopscience.iop.org' + issue
    issue_ = driver.get(issue_url)
    issue_content = driver.page_source
    
    abstracts = SoupStrainer(class_='art-list-item-title-wrapper')
    abstract_soup = BeautifulSoup(issue_content, 'html.parser', parse_only = abstracts)
    
    for link in abstract_soup.find_all('a'):
        title_link.append(link.get('href'))
    for title in abstract_soup.find_all('a'):
        titles.append(title.get_text())
    time.sleep(5)
driver.close()

In [25]:
for i in titles:
    print(i)


					Effects of surfaces and macromolecular crowding on bimolecular reaction rates

					Correlation between protein concentration and bacterial cell size can reveal mechanisms of gene expression

					From reversible to irreversible bistable switches via bifurcations in a gene regulatory network

					Quantifying molecular tension—classifications, interpretations and limitations of force sensors

					Constraining the complexity of promoter dynamics using fluctuations in gene expression

					An equilibrium model for ribosome competition

					Mapping spatio-temporal dynamics of single biomolecules in living cells

					Power series method for solving TASEP-based models of mRNA translation

					Finite element modelling of discontinuous action potential propagation in larval zebrafish and human cardiac tissue

					Salt bridge impact on global rigidity and thermostability in thermophilic citrate synthase

					Role of hydrodynamic interactions in chemotaxis of bacterial populations



In [27]:
import json
article_link = dict(zip(titles, title_link))
with open('Physical Biology.json', 'w+') as fp:
    json.dump(article_link, fp, indent = "\n")